In [22]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [23]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [24]:
ak.num(events.Jet)

<Array [8, 13, 18, 7, 13, ... 9, 16, 15, 19] type='297347 * int64'>

In [59]:
ak.sum(ak.num(events.Jet.pt))

3567289

In [62]:
print(events.Electron.cutBased.__doc__)
print("Total Number of Electrons: ", len(events.Electron.cutBased))
sum = 0

#fail
e = events.Electron[events.Electron.cutBased == 0]
length_fail = ak.sum(ak.num(e))
sum += length_fail
print(length_fail)

#veto
e = events.Electron[events.Electron.cutBased == 1]
length_veto = ak.sum(ak.num(e))
sum += length_veto
print(length_veto)

#loose
e = events.Electron[events.Electron.cutBased == 2]
length_loose = ak.sum(ak.num(e))
sum += length_loose
print(length_loose)

#medium
e = events.Electron[events.Electron.cutBased == 3]
length_medium = ak.sum(ak.num(e))
sum += length_medium
print(length_medium)

#tight
e = events.Electron[events.Electron.cutBased == 4]
length_tight = ak.sum(ak.num(e))
sum += length_tight
print(length_tight)

print("Total Sum of ID candidates = " , sum)
print("loose + medium + tight = " , length_loose+length_medium+length_tight)

cut-based ID Fall17 V2 (0:fail, 1:veto, 2:loose, 3:medium, 4:tight)
Total Number of Electrons:  297347
101532
18610
27066
25138
184587
Total Sum of ID candidates =  356933
loose + medium + tight =  236791


In [76]:
ak.sum(ak.num(events.Electron[events.Electron.cutBased >= 2 ] ))

236791